# Resampling Methods - Lab

## Introduction

Now that you have some preliminary background on bootstrapping, jacknife and permutation tests, its time to practice those skills by coding them into functions. You'll then apply these tests to a hypothesis test and compare the results to a parametric t-test.

## Objectives

You will be able to:
* Understand permutation testing
* Understand what jacknife is
* Understand what bootstrapping is

## Bootstrapping

Write a function that takes a sample and generates n additional samples of the same size using bootstrapping. (Recall that bootstrapping creates additional sets by sampling with replacement.)

In [72]:
import numpy as np

In [73]:
def bootstrap(sample, n_additional_samples):
    new_samples = []
    counter = 0
    while counter < n_additional_samples:
        new_samples.append(np.random.choice(sample, size = len(sample), replace = True))
        counter += 1
    return new_samples                   
                           

## Jacknife 

Write a function that creates additional samples by removing one element at a time. The function should do this for each of the n items in the original sample, returning n samples, each with n-1 members.

In [74]:
def jack1(sample):
    """This function should take in a list of n observations and return n lists
    each with one member (presumably the nth) removed."""
    # Your code here
    new_samples = []
    new_sample = sample
    new_sample.pop()
    while len(new_sample) > 0:
        new_samples.append(new_sample)
        new_sample.pop()
    return new_samples

## Permutation Testing

Define a function that generate all possible, equally sized, two set splits of two sets A and B. Sets A and B need not be the same size, but all of the generate two set splits should be of equal size. For example, if we had a set with 5 members and a set with 7 members, the function would return all possible 5-7 splits of the 12 items. 


Here's a more in depth example:  
```A = [1,2,2]
B = [1,3]
permT(A, B) = [
                ([1,2,2], [1,3]),
                ([1,2,3], [1,2]),
                ([1,2,1], [2,3])
                ([1,1,3], [2,2]),
                ([2,2,3], [1,1])
              ]```  
These are all the possible 3-2 member splits of the 5 elements : 1,1,2,2,3.

In [75]:
from itertools import permutations
from itertools import combinations

In [76]:
def permT(a,b):
    size_1 = len(a)
    size_2 = len(b)
    items = a + b
    copy = items
    new_a = []
    new_b = list(set(combinations(items, len(b))))
    for index in range(len(new_b)):
        new_b[index] = list(np.sort(new_b[index]))
    for item in new_b:
        count = 0
        for i in range(len(new_b)):
            if item == new_b[i]:
                count += 1
        if count > 1:
            new_b.remove(item)       
    #new_b is completed
    
    for item in new_b:
        copy_new = a + b
        for i in range(len(item)):
            x = item[i]
            copy_new.remove(x)
        new_a.append(copy_new)
    
    new = []
    for i in range(len(new_b)):
        permutation = (new_a[i], new_b[i])
        new.append(permutation)
    return new
    
    
    # Your code here

In [77]:
A = [1,2,2]
B = [1,3]
permT(A,B)

[([2, 2, 1], [1, 3]),
 ([2, 1, 3], [1, 2]),
 ([1, 2, 1], [2, 3]),
 ([1, 1, 3], [2, 2]),
 ([2, 2, 3], [1, 1])]

## Permutation Testing in Practice
Let's further investigate the scenario proposed in the previous lesson. Below are two samples A and B. The samples are mock data for the blood pressure of sample patients. The research study is looking to validate whether there is a statistical difference in the blood pressure of these two groups using a 5% signifincance level.  First, calculate the mean blood pressure of each of the two samples. Then, calculate the difference of these means. From there, use your `permT()` function, defined above, to generate all the possible combinations of the entrie sample data into A-B splits of equivalent sizes as the original sets. For each of these combinations, calculate the mean blood pressure of the two groups and record the difference between these sample means. The full collection of the difference in means between these generated samples will serve as the denominator to calculate the p-value associated with the difference between the original sample means.

For example, in our small handwritten example above:

$\mu_a = \frac{1+2+2}{3} = \frac{5}{3}$  
and  
$\mu_b = \frac{1+3}{2} = \frac{4}{2} = 2$  

Giving us

$\mu_a - \mu_b = \frac{5}{3} - 2 = \frac{1}{2}$

In comparison, for our various combinations we have:

([1,2,2], [1,3]):  $\mu_a - \mu_b = \frac{5}{3} - 2 = \frac{1}{2}$  
([1,2,3], [1,2]):  $\mu_a - \mu_b = 2 - \frac{3}{2} = \frac{1}{2}$  
([1,2,1], [2,3]):  $\mu_a - \mu_b = \frac{4}{3} - \frac{5}{3} = -\frac{1}{2}$  
([1,1,3], [2,2]):  $\mu_a - \mu_b = \frac{5}{3} - 2 = \frac{1}{2}$  
([2,2,3], [1,1]):  $\mu_a - \mu_b = \frac{7}{3} - 1 = \frac{4}{3}$  

A standard hypothesis test for this scenario might be:

$h_0: \mu_a = \mu_b$  
$h_1: \mu_a < \mu_b$  
  
Thus comparing our sample difference to the differences of our possible combinations, we look at the number of experiments from our permutation space that were the same or greater then our sample statistic, divided by the total number of permutations. In this case, 4 out of 5 of the permutation cases produced the same or greater differences in the two sample means. This value .8 is a strong indication that we cannot refute the null hypothesis for this instance.



In [78]:
# Your code here
a = A
b = B
sample_mean_difference= np.array(a).mean() - np.array(b).mean()
matrix = permT(a,b)
mean_differences = []
for item in matrix:
    first = np.array(item[0])
    second = np.array(item[1])
    first_mean = first.mean()
    second_mean = second.mean()
    diff = first_mean - second_mean
    mean_differences.append(diff)
mean_differences = np.array(mean_differences)
count = 0
for value in mean_differences:
    if sample_mean_difference < value:
        count += 1
p = count / len(mean_differences)
a_val = .05
if p > a_val:
    print ('cannot reject null hypothesis')
    print(p)
else:
    print('null hypothesis rejected')
    print(p)

cannot reject null hypothesis
0.4


## T-test Revisited

The parameteric statistical test equivalent to our permutation test above would be a t-test of the two groups. Perform a t-test on the same data above in order to calculate the p-value. How does this compare to the above results?

In [79]:
# Your code here
import scipy.stats as stats

In [80]:
a = A
b = B
mean_difference = sample_mean_difference
n_a = len(a)
n_b = len(b)
s_a = np.var(a)
s_b = np.var(b)
denom = np.sqrt((s_a / n_a) + (s_b / n_b))
print(denom)
t = mean_difference  / denom
p = stats.t.sf(np.abs(t), n_a + n_b - 1) * 2
p

0.7576767609436587


0.6827060250409898

## Bootstrap Applied

Use your code above to apply the bootstrap technique to this hypothesis testing scenario. In other words, similar to the permutation testing you performed above, compute additional samples (arbitrarily let's say 1000) of the same size as the original sample, with replacement. For each of these additional samples, compute whether the difference in sample means is the same or greater then that of the original samples. Use this to calculate an overall p-value for the null hypothesis.

In [88]:
# Your code here
count = 0
n_additional_samples = 1000
for i in range(n_additional_samples):
    new_a = bootstrap(a, len(a))
    new_b = bootstrap(b, len(b))
    new_diff = np.mean(new_a) - np.mean(new_b)
    if new_diff >= mean_difference:
        count += 1
p = count / n_additional_samples
p
    

0.556

## Summary

Well done! In this lab you practice coding modern statistical resampling techniques of the 20th century! You also started to compare these non-parametric methods to other parametric methods such as the t-test that we previously discussed.